In [1]:
#dependencies
import pandas as pd
import numpy as np
import json 
import requests

from config import govkey

### Stations vs AGI

In [2]:
#make paths
stn_path = 'Resources/stations.csv'
income_path = 'Resources/income.csv'

#read into df
stn_df = pd.read_csv(stn_path)
income_df = pd.read_csv(income_path)

#add leading 0s in the zip codes
stn_df['Zip Code'] = stn_df['Zip Code'].astype(str).str.zfill(5)
income_df['Zip Code'] = income_df['Zip Code'].astype(str).str.zfill(5)

stn_df.head()

,Station Name,Street Address,City,State,Zip Code,Connector1,Connector2,Connector3,Connector4,Geocode Status,Latitude,Longitude,ID
0,Lou Sobh Kia,1135 Buford Rd,Cumming,GA,30041,NaN,NaN,2.0,NaN,GPS,34.175783,-84.127427,74164
1,Lincoln Property,733 Concord Ave,Cambridge,MA,02138,NaN,2.0,NaN,NaN,GPS,42.391193,-71.153372,74170
2,Wisconsin Place Community Recreation Center,5307 Friendship Blvd,Bethesda,MD,20815,NaN,2.0,NaN,NaN,GPS,38.960782,-77.088105,81151
3,Performance Kia,229 SW Everett Mall Way,Everett,WA,98204,NaN,NaN,2.0,NaN,GPS,47.906081,-122.236582,74196
4,Lee Johnson Kia,11845 NE 85th St,Kirkland,WA,98033,NaN,NaN,2.0,NaN,GPS,47.678618,-122.181376,76253


In [3]:
income_df.head()

,State,Zip Code,Returns Filed,AGI
0,AK,99901,6420,411668
1,AK,99835,4570,303947
2,AK,99833,1670,98525
3,AK,99827,1300,74437
4,AK,99824,1180,87423


In [4]:
#function to get the numbers out of the string and sum them
def num_sum(string):
    nums = [int(s) for s in string.split() if s.isdigit()] 
    summed = 0
    for num in nums:
        summed += num
    return summed

In [5]:
#make a new column containing the summed #s from the string
for index, row in stn_df.iterrows():
    try:
        stn_df.at[index, 'Cnct 4 Summed'] = num_sum((row['Connector4']))
    except(AttributeError):
        pass
    
#double check and they are all numbers
stn_df['Cnct 4 Summed'].describe()

count      59.000000
mean       35.474576
std       260.304553
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max      2001.000000
Name: Cnct 4 Summed, dtype: float64

In [6]:
#Change all NaNs to 0
stn_df = stn_df.fillna(0)

#add all the connectors together and put in one column
stn_df['Number of Connectors'] = stn_df['Connector1'] + stn_df['Connector2'] + stn_df['Connector3'] + stn_df['Cnct 4 Summed']


print(stn_df.shape)    
stn_df.head()

(17686, 15)


,Station Name,Street Address,City,State,Zip Code,Connector1,Connector2,Connector3,Connector4,Geocode Status,Latitude,Longitude,ID,Cnct 4 Summed,Number of Connectors
0,Lou Sobh Kia,1135 Buford Rd,Cumming,GA,30041,0.0,0.0,2.0,0,GPS,34.175783,-84.127427,74164,0.0,2.0
1,Lincoln Property,733 Concord Ave,Cambridge,MA,02138,0.0,2.0,0.0,0,GPS,42.391193,-71.153372,74170,0.0,2.0
2,Wisconsin Place Community Recreation Center,5307 Friendship Blvd,Bethesda,MD,20815,0.0,2.0,0.0,0,GPS,38.960782,-77.088105,81151,0.0,2.0
3,Performance Kia,229 SW Everett Mall Way,Everett,WA,98204,0.0,0.0,2.0,0,GPS,47.906081,-122.236582,74196,0.0,2.0
4,Lee Johnson Kia,11845 NE 85th St,Kirkland,WA,98033,0.0,0.0,2.0,0,GPS,47.678618,-122.181376,76253,0.0,2.0


In [7]:
#multiply the agi column by 1000 (because it's in thousands of dollars)
income_df['AGI'] = income_df['AGI'] * 1000

#find the average agi
income_df['Average AGI'] = round((income_df['AGI'] / income_df['Returns Filed']), 2)

income_df.head()

,State,Zip Code,Returns Filed,AGI,Average AGI
0,AK,99901,6420,411668000,64122.74
1,AK,99835,4570,303947000,66509.19
2,AK,99833,1670,98525000,58997.01
3,AK,99827,1300,74437000,57259.23
4,AK,99824,1180,87423000,74087.29


In [8]:
#narrow down both dfs in prep for merging
nrw_stn_df = stn_df[['ID', 'State', 'Zip Code', 'Latitude', 'Longitude', 'Number of Connectors']]

nrw_income_df = income_df[['State', 'Zip Code', 'Average AGI']]

nrw_stn_df.head()

,ID,State,Zip Code,Latitude,Longitude,Number of Connectors
0,74164,GA,30041,34.175783,-84.127427,2.0
1,74170,MA,02138,42.391193,-71.153372,2.0
2,81151,MD,20815,38.960782,-77.088105,2.0
3,74196,WA,98204,47.906081,-122.236582,2.0
4,76253,WA,98033,47.678618,-122.181376,2.0


In [9]:
nrw_income_df.head()

,State,Zip Code,Average AGI
0,AK,99901,64122.74
1,AK,99835,66509.19
2,AK,99833,58997.01
3,AK,99827,57259.23
4,AK,99824,74087.29


In [10]:
#merge the two on zip code keeping everything
inc_stn_df = pd.merge(nrw_income_df, nrw_stn_df, on='Zip Code', how='outer')

inc_stn_df.head()

,State_x,Zip Code,Average AGI,ID,State_y,Latitude,Longitude,Number of Connectors
0,AK,99901,64122.74,NaN,NaN,NaN,NaN,NaN
1,AK,99835,66509.19,NaN,NaN,NaN,NaN,NaN
2,AK,99833,58997.01,NaN,NaN,NaN,NaN,NaN
3,AK,99827,57259.23,NaN,NaN,NaN,NaN,NaN
4,AK,99824,74087.29,NaN,NaN,NaN,NaN,NaN


In [11]:
#rename the columns
inc_stn_df.columns = ['AGI State', 'Zip Code', 'Average AGI', 'Stn ID', 'Stn State', 'Stn Lat', 'Stn Lng', 'Connectors']

inc_stn_df.head(6)

,AGI State,Zip Code,Average AGI,Stn ID,Stn State,Stn Lat,Stn Lng,Connectors
0,AK,99901,64122.74,NaN,NaN,NaN,NaN,NaN
1,AK,99835,66509.19,NaN,NaN,NaN,NaN,NaN
2,AK,99833,58997.01,NaN,NaN,NaN,NaN,NaN
3,AK,99827,57259.23,NaN,NaN,NaN,NaN,NaN
4,AK,99824,74087.29,NaN,NaN,NaN,NaN,NaN
5,AK,99801,70878.06,81601.0,AK,58.298366,-134.404006,2.0


In [12]:
#a few of the same zip codes have different states from each csv

#make a new df in preparation for fixing the differing states
states_df = inc_stn_df[['AGI State', 'Zip Code', 'Stn State']]

states_df.head(6)

,AGI State,Zip Code,Stn State
0,AK,99901,NaN
1,AK,99835,NaN
2,AK,99833,NaN
3,AK,99827,NaN
4,AK,99824,NaN
5,AK,99801,AK


In [13]:
#find where the states differ
states_df = states_df.dropna(axis=0, how='any')

#make a list to hold all that differ
dif_sts_lst = []

#loop through to find which ones differ
for index, row in states_df.iterrows():
    if row['AGI State'] != row['Stn State']:
        print(index, row['Zip Code'], row['AGI State'], row['Stn State'])
        dif_sts_lst.append(index)
        
print(dif_sts_lst)

9089 80109 CO NV
9090 80109 CO NV
9091 80109 CO NV
9092 80109 CO NV
9093 80109 CO NV
9094 80109 CO NV
9095 80109 CO NV
14006 66105 KS MO
20236 49747 MI ME
26299 33316 FL AL
26726 32803 FL GA
30033 25702 WV NC
[9089, 9090, 9091, 9092, 9093, 9094, 9095, 14006, 20236, 26299, 26726, 30033]


In [14]:
# https://www.unitedstateszipcodes.org/
#the income states are the correct ones so replace station state with them
for i in dif_sts_lst:
    inc_stn_df.loc[i, 'Stn State'] = inc_stn_df.loc[i, 'AGI State']

#check one    
inc_stn_df.loc[30033,]

AGI State           WV
Zip Code         25702
Average AGI    36980.8
Stn ID           61816
Stn State           WV
Stn Lat        35.7318
Stn Lng       -78.8544
Connectors           5
Name: 30033, dtype: object

In [15]:
#get just the zips with stations and put in a new df
stn_agi_df = inc_stn_df.dropna(subset=['Stn ID'], axis=0)

print(stn_agi_df.shape)
stn_agi_df.head()

(17686, 8)


,AGI State,Zip Code,Average AGI,Stn ID,Stn State,Stn Lat,Stn Lng,Connectors
5,AK,99801,70878.06,81601.0,AK,58.298366,-134.404006,2.0
6,AK,99801,70878.06,82228.0,AK,58.367717,-134.603090,1.0
7,AK,99801,70878.06,85589.0,AK,58.363473,-134.575083,1.0
8,AK,99801,70878.06,64702.0,AK,58.361350,-134.577297,1.0
9,AK,99801,70878.06,72524.0,AK,58.353653,-134.495592,2.0


In [16]:
#not all station zips have a corresponding agi

#find the stations without AGIs and put in a df
stn_no_agi_df = stn_agi_df[pd.isnull(stn_agi_df['Average AGI'])]

print(stn_no_agi_df.shape)
stn_no_agi_df.head()

(522, 8)


,AGI State,Zip Code,Average AGI,Stn ID,Stn State,Stn Lat,Stn Lng,Connectors
38842,NaN,91330,NaN,21395.0,CA,34.248959,-118.523887,2.0
38843,NaN,91330,NaN,21396.0,CA,34.241687,-118.528543,2.0
38844,NaN,91330,NaN,21397.0,CA,34.236853,-118.532588,4.0
38845,NaN,91330,NaN,54076.0,CA,34.242412,-118.524740,2.0
38846,NaN,91330,NaN,66721.0,CA,34.241917,-118.532194,2.0


In [17]:
#read the nearest zip into a df
nz_df = pd.read_csv('Resources/nearestzip.csv')

print(nz_df.shape)

#narrow zip1 column down to only the zips we are looking for
nz_df = nz_df[nz_df['zip1'].isin(stn_no_agi_df['Zip Code'])]
print(nz_df.shape)

#narrow zip2 column down to only the zips we have agis for
nz_df = nz_df[nz_df['zip2'].isin(income_df['Zip Code'])]
print(nz_df.shape)

#rename columns
nz_df.columns = ['orig_zip', 'new_zip', 'distance_miles']

#keep only the first zip because it's the closest and put in a new df
nz_single_df = nz_df.drop_duplicates(['orig_zip'], keep='first')

nz_single_df.shape

(1873912, 3)
(11829, 3)
(10388, 3)


(162, 3)

In [18]:
#rename columns
nz_single_df = nz_single_df.rename(columns={"orig_zip": "Old Zip", "new_zip": "Zip Code", "distance_miles": "Nearest Zip Distance"})

#add leading zeroes
nz_single_df['Old Zip'] = nz_single_df['Old Zip'].astype(str).str.zfill(5)
nz_single_df['Zip Code'] = nz_single_df['Zip Code'].astype(str).str.zfill(5)

#save as csv for github since the original is too large
nz_single_df.to_csv('Resources/nz_single_df.csv')

nz_single_df.head()

,Old Zip,Zip Code,Nearest Zip Distance
6558,01003,01035,3.241487
86373,03575,03593,5.162231
101900,04469,04473,3.694574
104883,04662,04679,2.207962
109025,05031,05067,4.378147


In [19]:
#merge new zips with AGI data
stn_dropnoagi_df = stn_agi_df.dropna(subset=['Average AGI'], axis=0)

zip_combined_df = pd.merge(nz_single_df, nrw_income_df, on="Zip Code", how="left")

zip_combined_df = zip_combined_df[["Zip Code", "Nearest Zip Distance", "State", "Average AGI"]]

#merge new AGI data with overall AGI data to get complete dataset
stn_pre_agi_df = pd.merge(stn_dropnoagi_df, zip_combined_df, on="Zip Code", how="left")

stn_pre_agi_df = stn_pre_agi_df.rename(columns={"Average AGI_x": "Average AGI"})

stn_pre_agi_df = stn_pre_agi_df[["AGI State","Stn State", "Zip Code", "Average AGI", "Stn ID",
                                "Stn Lat", "Stn Lng", "Connectors", "Nearest Zip Distance", "State"]]

#drop AGIs without station IDs
stn_pre_agi_df = stn_pre_agi_df.dropna(subset=["Stn ID"])

stn_pre_agi_df.to_csv("Resources/station_agi_data.csv")

In [20]:
#group dataset on zip code and station ID to prepare final datafame for plotting
stn_group = stn_pre_agi_df.groupby(["Zip Code", "Stn ID"])


In [21]:
#clean up column heads
station_agi_df = stn_pre_agi_df[["AGI State", "Stn State", "Zip Code", "Average AGI",
                                "Stn ID", "Stn Lat", "Stn Lng", "Connectors", "Nearest Zip Distance", "State"]]

#add a formatted agi column for the hover text in the map
for index,row in station_agi_df.iterrows():
    station_agi_df.at[index, 'formatted_agi'] = ('${:,.2f}'.format(row['Average AGI']))

#station_agi_df.head(7)

#export for use in a map
station_agi_df.to_csv('Output/station_agi_df.csv')

station_agi_df.head()

,AGI State,Stn State,Zip Code,Average AGI,Stn ID,Stn Lat,Stn Lng,Connectors,Nearest Zip Distance,State,formatted_agi
0,AK,AK,99801,70878.06,81601.0,58.298366,-134.404006,2.0,NaN,NaN,"$70,878.06"
1,AK,AK,99801,70878.06,82228.0,58.367717,-134.603090,1.0,NaN,NaN,"$70,878.06"
2,AK,AK,99801,70878.06,85589.0,58.363473,-134.575083,1.0,NaN,NaN,"$70,878.06"
3,AK,AK,99801,70878.06,64702.0,58.361350,-134.577297,1.0,NaN,NaN,"$70,878.06"
4,AK,AK,99801,70878.06,72524.0,58.353653,-134.495592,2.0,NaN,NaN,"$70,878.06"


In [22]:
#begin creating dataframe for number of stations by AGI plot

#group by zip code
stn_group = station_agi_df.groupby(["Zip Code"])

#find number of stations per zip code
stn_group_df = pd.DataFrame(station_agi_df["Zip Code"].value_counts())

#find average AGI per zip code
stn_group_df["Average AGI"] = stn_group["Average AGI"].mean()

#find number of connectors per zip code
stn_group_df["Number of Connectors"] = stn_group["Connectors"].sum()

stn_group_df = stn_group_df.reset_index()

stn_group_df = stn_group_df.rename(columns={"index": "Zip Code", "Zip Code": "Number of Stations"})

stn_group_df.head()

#stn_group_df.to_csv('stn_group_df.csv')

,Zip Code,Number of Stations,Average AGI,Number of Connectors
0,92618,47,122389.22,178.0
1,94025,44,354954.02,299.0
2,64105,44,57048.15,256.0
3,94538,42,83908.61,135.0
4,95814,41,62561.27,215.0


In [23]:
#bin the AGI

#make the bins and labels 
agi_bins = [0, 49999.90, 74999.90, 99999.90, 124999.90, 149999.90, 174999.99, 99999999]
bin_names = ["<50k", "50k-75k", "75k-100k", "100k-125k", "125k-150k", "150k-175k", "175k+"]

#copy into a new df
stn_agi_bin_df = stn_group_df.copy(deep=True)

#put agi into the bins 
stn_agi_bin_df["AGI Range"] = pd.cut(stn_group_df["Average AGI"], agi_bins, labels=bin_names)

#stn_agi_bin_df.to_csv('bintest.csv')
stn_agi_bin_df.head()

,Zip Code,Number of Stations,Average AGI,Number of Connectors,AGI Range
0,92618,47,122389.22,178.0,100k-125k
1,94025,44,354954.02,299.0,175k+
2,64105,44,57048.15,256.0,50k-75k
3,94538,42,83908.61,135.0,75k-100k
4,95814,41,62561.27,215.0,50k-75k


In [25]:
#iterate through binned dataframe in order to append percentage of total zips to each row

for index, row in stn_agi_bin_df.iterrows():
    
    if row["AGI Range"] == "<50k":
        
        stn_agi_bin_df.set_value(index, "Percentage of Total Zips", 28.25)
    
    elif row["AGI Range"] == "50k-75k":
        
        stn_agi_bin_df.set_value(index, "Percentage of Total Zips", 37.00)
    
    elif row["AGI Range"] == "75k-100k":
        
        stn_agi_bin_df.set_value(index, "Percentage of Total Zips", 15.33)
        
    elif row["AGI Range"] == "100k-125k":
        
        stn_agi_bin_df.set_value(index, "Percentage of Total Zips", 7.38)
        
    elif row["AGI Range"] == "125k-150k":
        
        stn_agi_bin_df.set_value(index, "Percentage of Total Zips", 3.78)
        
    elif row["AGI Range"] == "150k-175k":
        
        stn_agi_bin_df.set_value(index, "Percentage of Total Zips", 1.78)
        
    elif row["AGI Range"] == "175k+":
        
        stn_agi_bin_df.set_value(index, "Percentage of Total Zips", 5.88)

#export to csv for plotting
stn_agi_bin_df = stn_agi_bin_df.sort_values(by="AGI Range", ascending=False)
stn_agi_bin_df.to_csv('Output/stn_agi_bin_df.csv')
stn_agi_bin_df.head()

,Zip Code,Number of Stations,Average AGI,Number of Connectors,AGI Range,Percentage of Total Zips
1676,10005,3,614329.55,5.0,175k+,5.88
4755,20015,1,231809.22,6.0,175k+,5.88
2567,77019,2,377515.50,4.0,175k+,5.88
1038,08540,4,219322.34,7.0,175k+,5.88
3510,11797,1,266937.91,2.0,175k+,5.88


In [26]:
#create dataframe on binned AGI values
stn_agi_bin_grp = stn_agi_bin_df.groupby('AGI Range')
stn_agi_bin_grp.max()

,Zip Code,Number of Stations,Average AGI,Number of Connectors,Percentage of Total Zips
AGI Range,,,,,
<50k,99336,21,49999.42,105.0,28.25
50k-75k,99801,44,74999.73,256.0,37.00
75k-100k,99669,42,99973.96,2077.0,15.33
100k-125k,98642,47,124894.92,178.0,7.38
125k-150k,98119,29,149912.39,118.0,3.78
150k-175k,98121,39,174769.19,492.0,1.78
175k+,98164,44,1496505.85,299.0,5.88


In [27]:
#create dataframe on grouped AGI bins for final dataframe
stn_agi_count_df = pd.DataFrame(stn_agi_bin_df['AGI Range'].value_counts())
stn_agi_count_df = stn_agi_count_df.reset_index()
stn_agi_count_df = stn_agi_count_df.rename(columns={"index": "AGI Range", "AGI Range": "Number of Zipcodes"})
stn_agi_count_df = stn_agi_count_df.sort_values(by="AGI Range", ascending=True)
stn_agi_count_df = stn_agi_count_df.set_index("AGI Range")
stn_agi_count_df["Number of Stations"] = stn_agi_bin_grp["Number of Stations"].sum()
stn_agi_count_df["Number of Connectors"] = stn_agi_bin_grp["Number of Connectors"].sum()
total_zipcodes = stn_agi_count_df["Number of Zipcodes"].sum()

stn_agi_count_df["Percentage of Total Zips"] = round((stn_agi_count_df["Number of Zipcodes"]/total_zipcodes)*100, 2)
stn_agi_count_df = stn_agi_count_df[["Number of Stations", "Number of Connectors", "Percentage of Total Zips"]]
stn_agi_count_df = stn_agi_count_df.reset_index()
stn_agi_count_df.head(7)

,AGI Range,Number of Stations,Number of Connectors,Percentage of Total Zips
0,<50k,3668,9967.0,28.85
1,50k-75k,5610,14843.0,37.00
2,75k-100k,2971,10022.0,15.33
3,100k-125k,1629,4603.0,7.38
4,125k-150k,915,2509.0,3.78
5,150k-175k,479,1783.0,1.78
6,175k+,1922,5551.0,5.88


### Stations vs Incentives

In [28]:
#API Calls for incentives data

state_id = []

count = 1

#append states to state_id list
for index, row in station_agi_df.iterrows():
    
    if row["Stn State"] not in state_id:
        
        state_id.append(row["Stn State"])
        
        
state_df = pd.DataFrame({"State": state_id})        

count = 1

#run API requests
for index, row in state_df.iterrows():
    
    print("Now fetching state #%s of %s" % (count, len(state_df["State"])))
    
    incentives_url = "https://developer.nrel.gov/api/transportation-incentives-laws/v1.json?api_key=" + govkey + "&limit=100&jurisdiction=" + row["State"] + "&technology=ELEC&incentive_type=GNT%2CTAX%2CLOANS%2CRBATE%2CEXEM&regulation_type=REGIS%2CSTD%2CDREST%2CREQ%2CFUEL%2CRFS%2CAIRQEMISSIONS%2COTHER%2CCCEINIT&user_type=IND%2CFLEET%2CSTATION%2CAFP%2CAFS%2CPURCH%2CMAN%2COTHER&poc=false&recent=false&expired=false&law_type=STATEINC%2CUPINC%2CLAWREG%2CINC%2CPROG&local=false"
    
    incentives = requests.get(incentives_url)
    
    incentives_json = incentives.json()
    
    state_df.set_value(index, "Number of Incentives", len((incentives_json["result"])))
    
    count += 1


Now fetching state #1 of 51
Now fetching state #2 of 51
Now fetching state #3 of 51
Now fetching state #4 of 51
Now fetching state #5 of 51
Now fetching state #6 of 51
Now fetching state #7 of 51
Now fetching state #8 of 51
Now fetching state #9 of 51
Now fetching state #10 of 51
Now fetching state #11 of 51
Now fetching state #12 of 51
Now fetching state #13 of 51
Now fetching state #14 of 51
Now fetching state #15 of 51
Now fetching state #16 of 51
Now fetching state #17 of 51
Now fetching state #18 of 51
Now fetching state #19 of 51
Now fetching state #20 of 51
Now fetching state #21 of 51
Now fetching state #22 of 51
Now fetching state #23 of 51
Now fetching state #24 of 51
Now fetching state #25 of 51
Now fetching state #26 of 51
Now fetching state #27 of 51
Now fetching state #28 of 51
Now fetching state #29 of 51
Now fetching state #30 of 51
Now fetching state #31 of 51
Now fetching state #32 of 51
Now fetching state #33 of 51
Now fetching state #34 of 51
Now fetching state #35 

In [29]:
state_df.head()

,State,Number of Incentives
0,AK,1.0
1,WA,17.0
2,OR,19.0
3,HI,8.0
4,CA,57.0


In [30]:
#create incentives dataframe

state_df = state_df.rename(columns={"State": "Stn State"})

state_df.set_index("Stn State")

#merge station data with state data
station_state_df = pd.merge(station_agi_df, state_df, on="Stn State", how="left")

#group state values together to get sum of connectors 
station_connectorgroup_df = station_state_df.groupby("Stn State")

station_stategroup_df = station_state_df.groupby(["Stn State", "Number of Incentives"])

station_stategroup_df = pd.DataFrame(station_stategroup_df.size().reset_index())

station_stategroup_df = station_stategroup_df.rename(columns={"Stn State": "State", 0: "Number of Stations"})

station_stategroup_df = station_stategroup_df.set_index("State")

station_stategroup_df["Number of Connectors"] = station_connectorgroup_df["Connectors"].sum()

station_stategroup_df = station_stategroup_df.reset_index()


station_stategroup_df.head()

,State,Number of Incentives,Number of Stations,Number of Connectors
0,AK,1.0,7,11.0
1,AL,1.0,73,176.0
2,AR,3.0,45,94.0
3,AZ,15.0,389,987.0
4,CA,57.0,4119,16501.0


In [31]:
#make a column for the text color
station_stategroup_df['text_color'] = np.where(station_stategroup_df['Number of Incentives'] > 8, 'white', 'black')

#export for use in plots
station_stategroup_df.to_csv("Output/Incentives by State.csv")

station_stategroup_df.head()

,State,Number of Incentives,Number of Stations,Number of Connectors,text_color
0,AK,1.0,7,11.0,black
1,AL,1.0,73,176.0,black
2,AR,3.0,45,94.0,black
3,AZ,15.0,389,987.0,white
4,CA,57.0,4119,16501.0,white


### Stations vs Gas Tax

In [32]:
#read the gas tax csv into a df
tax_df = pd.read_csv("Gasoline.csv")

#change State tax to a float
tax_df['Total State'] = tax_df['Total State'].astype(float)

tax_df.head()

,State,Total State
0,AL,0.1900
1,AK,0.0895
2,AZ,0.1900
3,AR,0.2180
4,CA,0.4886


In [33]:
#merge gas_df with the incetives df
gas_df = pd.merge(station_stategroup_df, tax_df, on="State", how="left")

#rename state tax column
gas_df = gas_df.rename(columns={"Total State": "State Tax"})

#add the text color column for when plotting on the map
gas_df['text_color'] = np.where(gas_df['State Tax'] > .4, 'white', 'black')

#export for use in plotting
gas_df.to_csv('Output/gas_df.csv')

gas_df.head()

,State,Number of Incentives,Number of Stations,Number of Connectors,text_color,State Tax
0,AK,1.0,7,11.0,black,0.0895
1,AL,1.0,73,176.0,black,0.1900
2,AR,3.0,45,94.0,black,0.2180
3,AZ,15.0,389,987.0,black,0.1900
4,CA,57.0,4119,16501.0,white,0.4886
